In [1]:
import numpy
import datetime
import pandas as pd
from datetime import datetime
import vaex as vx

In [2]:
import matplotlib.pyplot as plt
from  plotly.offline import iplot
import plotly as py 
import plotly.express as px

In [3]:
import cufflinks as cf 
py.offline.init_notebook_mode(connected=True)
cf.go_offline()

In [4]:
full_data = pd.read_csv('full_data.csv')
full_data

,MESSAGE_ID,TRIP_ID,ROUTENAME,BUS_ID,ROUTE_ORIGIN_STOP,ROUTE_DESTINATION_STOP,TRIP_START_TIME,TRIP_END_TIME,AMOUNT,USER_COUNT,TICKET DATE,TICKET TIME
0,10748200719093707,T10748200719093605,101BDOWN,DL1PC5488,Swaroop Nagar JJ Colony,Old Delhi Railway Station,20-JUL-2019 09:36:45,20-JUL-2019 10:37:56,15,1,20/07/2019,09:37:13
1,10748200719094328,T10748200719093605,101BDOWN,DL1PC5488,Libas Pur GT Road,Old Secretariat (Postal Account Office),20-JUL-2019 09:36:45,20-JUL-2019 10:37:56,10,1,20/07/2019,09:43:29
2,10748200719093706,T10748200719093605,101BDOWN,DL1PC5488,Swaroop Nagar JJ Colony,Old Delhi Railway Station,20-JUL-2019 09:36:45,20-JUL-2019 10:37:56,15,1,20/07/2019,09:37:10
3,10748200719093708,T10748200719093605,101BDOWN,DL1PC5488,Swaroop Nagar JJ Colony,Old Delhi Railway Station,20-JUL-2019 09:36:45,20-JUL-2019 10:37:56,15,1,20/07/2019,09:37:15
4,10748200719093709,T10748200719093605,101BDOWN,DL1PC5488,Swaroop Nagar JJ Colony,Old Delhi Railway Station,20-JUL-2019 09:36:45,20-JUL-2019 10:37:56,15,1,20/07/2019,09:37:46
...,...,...,...,...,...,...,...,...,...,...,...,...
134291945,11680291219195309,T11680291219173602,TMS-PBAGH,DL1PC4277,Model Town II,Wazirpur Depot,29-DEC-2019 17:36:50,29-DEC-2019 20:02:40,5,1,29/12/2019,19:53:46
134291946,11132291219203421,T11132291219184502,TMS-PBAGH,DL1PC4326,Model Town II,Wazirpur Depot,29-DEC-2019 18:45:18,29-DEC-2019 20:53:04,5,1,29/12/2019,20:34:50
134291947,11680291219191147,T11680291219173602,TMS-PBAGH,DL1PC4277,Railway Road Bridge,Old Secretariat (Postal Account Office),29-DEC-2019 17:36:50,29-DEC-2019 20:02:40,10,1,29/12/2019,19:11:14
134291948,12278291219191122,T12278291219165502,TMS-PBAGH,DL1PC4127,Ashok Vihar Crossing,Punjabi Bagh Terminal,29-DEC-2019 16:55:34,29-DEC-2019 19:19:39,5,1,29/12/2019,19:11:21


In [5]:
from tqdm import tqdm

In [6]:
def convert_to_datetime(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d/%m/%Y %H:%M:%S'))

def convert_to_date(date_string):
    return numpy.datetime64(datetime.strptime(str(date_string), '%d/%m/%Y'))

In [10]:
from datetime import timedelta

temp_l = list(full_data['ROUTENAME'].unique())
temp_l = temp_l[331:]

for r in tqdm(temp_l):
    
    route_data_pd = full_data[full_data['ROUTENAME'] == r]
    
    route_data_pd['date_time']  = (route_data_pd['TICKET DATE'] + ' ' + route_data_pd['TICKET TIME']).apply(convert_to_datetime)
    route_data_pd['ticket_date']  = route_data_pd['TICKET DATE'].apply(convert_to_date)
    
    for t in route_data_pd['TRIP_ID'].unique():
        Stops_refer_latlong_distance = pd.read_csv(f'D:\\MTech\\Dissertation\\Data\\GTFS_with_trip_length\\{r}.csv',index_col='Unnamed: 0')
        Stops_refer_latlong_distance.rename(columns = {'Lat':'stop_lat','Lon':'stop_lon'}, inplace = True)
        
        one_trip = route_data_pd[route_data_pd['TRIP_ID']==t].sort_values(by= 'date_time')

        origin_stops_order_etm = {}
        for i in range(len(one_trip)):
            if one_trip.iloc[i,4] in origin_stops_order_etm.keys():
                continue
            else:
                origin_stops_order_etm[one_trip.iloc[i,4]] = [one_trip.iloc[i,12],one_trip[one_trip['ROUTE_ORIGIN_STOP']==one_trip.iloc[i,4]]['USER_COUNT'].count()]

        trip_stops_names = list(Stops_refer_latlong_distance['stop_name'])
        stop_order_trip = []
        for i in trip_stops_names:
            if i not in stop_order_trip:
                stop_order_trip.append(i)

        boarding_stops = list(origin_stops_order_etm.keys())
        boarding_time_count = list(origin_stops_order_etm.values())

        for i in range(len(boarding_stops)-1):

            stop_1_time = boarding_time_count[i][0]
            stop_1_dist = Stops_refer_latlong_distance[Stops_refer_latlong_distance['stop_name']==boarding_stops[i]]['Distance_from_start'].values[0]

            time_difference = boarding_time_count[i+1][0]-boarding_time_count[i][0]
            speed = (Stops_refer_latlong_distance[Stops_refer_latlong_distance['stop_name']==boarding_stops[1]]['Distance_from_start'].values[0] -\
                     Stops_refer_latlong_distance[Stops_refer_latlong_distance['stop_name']==boarding_stops[0]]['Distance_from_start'].values[0]) * 1000 /\
                    (time_difference.seconds)

            s = stop_order_trip.index(boarding_stops[i])
            e = stop_order_trip.index(boarding_stops[i+1])

            for j in range(s,e):

                stop_x = stop_order_trip[j]

                stop_x_dist = Stops_refer_latlong_distance[Stops_refer_latlong_distance['stop_name']==stop_order_trip[j]]['Distance_from_start'].values[0]
                stop_y_dist = Stops_refer_latlong_distance[Stops_refer_latlong_distance['stop_name']==stop_order_trip[j+1]]['Distance_from_start'].values[0]

                stop_x_time = stop_1_time + timedelta(seconds=(stop_x_dist - stop_1_dist) * 1000 / speed)
                stop_y_time = stop_1_time + timedelta(seconds=(stop_y_dist - stop_1_dist) * 1000 / speed)

                new_boarding_index = one_trip[(one_trip['date_time'] >= f'{stop_x_time}') &  (one_trip['date_time'] < f'{stop_y_time}')].index
                route_data_pd.loc[new_boarding_index,'ROUTE_ORIGIN_STOP'] = f'{stop_order_trip[j]}'
    route_data_pd.to_csv(f'D:\\MTech\\Dissertation\\Data\\boarding inferred data\\{r}.csv',index=False)
        

100%|████████████████████████████████████████████████████████████████████████████| 220/220 [32:22:49<00:00, 529.86s/it]
